### Script to generate FATES parameter files 
- take ensemble members and for each one duplicate and change vertical scaling parameters

In [1]:
from scipy.stats import qmc
import numpy as np

import csv
import xarray as xr
import pandas as pd
import os
import netCDF4 as nc4
import sys
import shutil
from tempfile import TemporaryFile                                                                                                                                 
import argparse                                                                                                                                                                                                                                                                                                       
import tempfile 
import random
import re
import math

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib import cm
import matplotlib.colors as mcolors
from matplotlib.colors import Normalize
from matplotlib.colors import BoundaryNorm


import functions_py.fates_xarray_functions as fa

import modp as mp


### Load the large ensemble and choose ensemble members with coexistence 

In [2]:
#  local met  drivers
fates = '/pscratch/sd/j/jneedham/elm_runs/bci/mar24/bci_local_ensemble_2pfts_0324.E74c582a640-Feff4c934.2024-03-23/run/bci_local_ensemble_2pfts_0324.E74c582a640-Feff4c934.2024-03-23.h0.ensemble.sofar.nc'
fates = xr.open_dataset(fates,  decode_times=False)
time = fates.variables['time'] / 365.
cohort_size_bins = fates.variables['fates_levscls'][:]
nsizebins = len(cohort_size_bins)

tave_start =  250
tave_stop = 300

ninst = 200

In [3]:
AGB_PF = fa.scpf_to_scls_by_pft(fates.FATES_VEGC_ABOVEGROUND_SZPF, fates)
AGB_PF = AGB_PF.sum(dim='fates_levscls')
AGB_PF = AGB_PF.isel(time=slice(tave_start, tave_stop)).mean(dim='time')

In [5]:
AGB_tot = AGB_PF.sum(dim='fates_levpft')
AGB_PF1 = AGB_PF.isel(fates_levpft=0)

ratio = np.squeeze(AGB_PF1/AGB_tot)

keep = np.where((ratio >= 0.05) & (ratio <= 0.95))

num = np.array(keep).size
print(num)

print(np.squeeze(keep))

keep = np.squeeze(keep) + 1

print(keep)

8
[  1  33  41  84 116 144 146 159]
[  2  34  42  85 117 145 147 160]


In [6]:
vals = [2.916, 2.43, 1.944, 1.458]


# for each ensemble member with some coexistence... 
for i in keep :
   
    input_fname = '/global/homes/j/jneedham/BCI_leafmr/param_dir/Ensemble_params_0324/fates_params_bci_ens_{0}.nc'.format(i)

    # for each of four vertical scaling schemes... 
    for j in range(0,4) : 
        fout = '/global/homes/j/jneedham/BCI_leafmr/param_dir/Rdark_vert_ensemble/fates_params_rdark_ens{0}_vert{1}.nc'.format(i,j)
    
        shutil.copyfile(input_fname, fout)                                                                                                                             
   
        var = 'fates_maintresp_leaf_decay'
        organ = 0
        val = vals[j]
        
        pft = 1
        mp.main(var = var, pft = pft, fin = fout, val = val, 
                fout = fout, O = 1, organ = organ)
        
        pft = 2
        mp.main(var = var, pft = pft, fin = fout, val = val, 
                fout = fout, O = 1, organ = organ)
            
        

In [8]:
print(2.43*0.6)
print(2.43 * 0.8)
print(2.43)
print(2.43 * 1.2)

1.458
1.9440000000000002
2.43
2.916
